In [93]:
from datetime import datetime, timedelta
from meteostat import Point, Daily
import yfinance as yf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [94]:
start = datetime(2021, 1, 1)
end = datetime(2024, 1, 1)

In [95]:
bg = pd.DataFrame(yf.Ticker("VITL").history(start=start,end=end)).reset_index()
bg['Dir'] = np.where(bg["Close"]>bg['Open'], 1, 0)
bg['Date'] = pd.to_datetime(bg['Date'])
bg.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Dir
0,2021-01-04 00:00:00-05:00,25.410000,25.440001,24.080000,25.059999,384800,0.0,0.0,0
1,2021-01-05 00:00:00-05:00,25.290001,25.820000,25.000999,25.629999,317100,0.0,0.0,1
2,2021-01-06 00:00:00-05:00,26.000000,27.469999,25.500000,26.549999,621200,0.0,0.0,1
3,2021-01-07 00:00:00-05:00,26.139999,27.650000,25.709999,27.530001,365200,0.0,0.0,1
4,2021-01-08 00:00:00-05:00,27.520000,28.500000,26.799999,27.160000,459400,0.0,0.0,0


Vital is a US agricultural powerhouse that sells eggs. Their biggest farms are in Missouri(39.0997, -94.5786, 256)

In [96]:
datetime.today().date()

datetime.date(2025, 9, 15)

In [97]:


br_loc = Point(39.0997, -94.5786, 256)
br_data = Daily(br_loc,start,end)
brDf = pd.DataFrame(br_data.fetch())
brDf.dropna()
brDf.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2021-01-01,-1.9,-3.5,-0.2,14.5,0.0,<NA>,15.7,<NA>,1016.1,<NA>
2021-01-02,-2.6,-4.1,0.4,0.0,5.0,<NA>,1.6,<NA>,1019.0,<NA>
2021-01-03,-1.8,-6.3,2.0,0.0,3.0,<NA>,7.6,<NA>,1016.2,<NA>
2021-01-04,1.4,-1.9,5.9,0.0,0.0,<NA>,7.4,<NA>,1017.2,<NA>
2021-01-05,2.2,-4.6,9.8,0.0,0.0,<NA>,4.7,<NA>,1020.0,<NA>


In [98]:
brDf = brDf.drop(["wdir",'wpgt', 'tsun'],axis=1).reset_index()

In [99]:
brDf.head()

,time,tavg,tmin,tmax,prcp,snow,wspd,pres
0,2021-01-01,-1.9,-3.5,-0.2,14.5,0.0,15.7,1016.1
1,2021-01-02,-2.6,-4.1,0.4,0.0,5.0,1.6,1019.0
2,2021-01-03,-1.8,-6.3,2.0,0.0,3.0,7.6,1016.2
3,2021-01-04,1.4,-1.9,5.9,0.0,0.0,7.4,1017.2
4,2021-01-05,2.2,-4.6,9.8,0.0,0.0,4.7,1020.0


In [100]:
brDf['time'] = brDf['time'].dt.tz_localize(None)
bg['Date'] = bg['Date'].dt.tz_localize(None)
data = pd.merge(brDf,bg.drop(['Close', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'],axis=1), left_on='time', right_on='Date')

In [101]:
data.head()

,time,tavg,tmin,tmax,prcp,snow,wspd,pres,Date,Open,Dir
0,2021-01-04,1.4,-1.9,5.9,0.0,0.0,7.4,1017.2,2021-01-04,25.410000,0
1,2021-01-05,2.2,-4.6,9.8,0.0,0.0,4.7,1020.0,2021-01-05,25.290001,1
2,2021-01-06,4.0,3.1,5.4,7.1,0.0,7.4,1019.4,2021-01-06,26.000000,1
3,2021-01-07,1.6,-0.2,3.1,0.3,0.0,14.2,1024.9,2021-01-07,26.139999,1
4,2021-01-08,-0.3,-3.0,3.1,0.0,0.0,7.7,1028.4,2021-01-08,27.520000,0


In [102]:
X = data[['Open', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wspd', 'pres']].values
y = data['Dir'].values

In [103]:
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [105]:
# Ensure input arrays are float32 for Keras compatibility
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(32))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5449 - loss: 0.6901 - val_accuracy: 0.4967 - val_loss: 0.7016
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5449 - loss: 0.6892 - val_accuracy: 0.4967 - val_loss: 0.7021
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6898 - val_accuracy: 0.4967 - val_loss: 0.7006
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6893 - val_accuracy: 0.4967 - val_loss: 0.7005
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6886 - val_accuracy: 0.4967 - val_loss: 0.6995
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6890 - val_accuracy: 0.4967 - val_loss: 0.6973
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6898 - val_accuracy: 0.4967 - val_loss: 0.6965
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5449 - loss: 0.6883 - val_accuracy: 0.4967 - val_loss

In [106]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4967 - loss: 0.6967 
Test Accuracy: 0.50
